# **ĐỒ ÁN: NHẬP MÔN KHOA HỌC DỮ LIỆU** #

### Nhóm 7 ###
|<div style="width:150px">MSSV</div>|<div style="width:290px">Tên</div>|
|---------- |:-------------:|
| 20120041  |  Trần Kim Bảo |
| 20120053  |    Nguyễn Thành Đạt   | 
| 20120071  | Nguyễn Thị Bích Hà |
| 20120113  | Lê Nguyên Khang |

## <font color='orange'> Import thư viện </font> ##

In [ ]:
!pip install scrapy
!pip install spider3

In [2]:
import json
import re
import pandas as pd
import numpy as np
import scrapy
from pandas.testing import assert_frame_equal # to compare two dataframes
# YOUR CODE HERE (OPTION) 
# Nếu cần các thư viện khác thì bạn có thể import thêm tại đây

## <font color='orange'> Mô tả trang web </font> ##

Trong đồ án này nhóm mình xin được giới thiệu đến mọi người danh sách phim hay nhất mọi thời đại *(Thời gian tính đến ngày 19/11/2022)* được trang IMDb tổng hợp. **IMDb** (<font color='blue'>https://www.imdb.com/</font>) là một trang cơ sở dữ liệu trực tuyến về điện ảnh thế giới, cung cấp thông tin về phim, diễn viên, đạo diễn, nhà làm phim,.. và tất cả những người, công ty trong lĩnh vực sản xuất phim, phim truyền hình và cả trò chơi video.

Những thông tin được thu thập tại website: <font color='blue'>https://www.imdb.com/list/ls006266261/</font>

### Tạo một project mới với scrapy ###

In [ ]:
!scrapy startproject film_crawler

In [ ]:
cd film_crawler/film_crawler

## <font color='orange'> Thu thập thông tin từ web </font> ##
**<h5>1. Thu thập url, id, votes, gross của các phim </h5>**

In [5]:
%%writefile spiders/collect_film_urls.py
import scrapy

class collect_film_url(scrapy.Spider):
  name='film_urls' 
  
  def start_requests(self):
    urls=['https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page=1']
    self.page=1
    yield scrapy.Request(url=urls[0], callback=self.parse)
        
  def parse(self, response):
    top=response.css('div.lister-list span.lister-item-index.unbold.text-primary::text').getall()
    id=response.css('div.lister-list img::attr(data-tconst)').getall()
    url=response.css('div.lister-list div.lister-item.mode-detail div.lister-item-image.ribbonize a::attr(href)').re(r'.*/title/.*\d{4,10}.')
    for i in range(len(top)):
      yield{
        'Top': top[i],
        'ID':id[i],
        'URL':url[i],
        'Votes':response.xpath(f'//*[@id="main"]/div/div[4]/div[3]/div[{i+1}]/div[2]/p[4]/span[2]/@data-value').get(),
        'Gross':response.xpath(f'//*[@id="main"]/div/div[4]/div[3]/div[{i+1}]/div[2]/p[4]/span[5]/@data-value').get()
      }
    if self.page<11:
      self.page+=1
      next_page_url=f'https://www.imdb.com/list/ls006266261/?sort=list_order,asc&st_dt=&mode=detail&page={self.page}'
      yield scrapy.Request(url=next_page_url, callback=self.parse)
      

Overwriting spiders/collect_film_urls.py


- **Xuất các thông tin đã thu thập ở trên ra file json**

In [ ]:
!scrapy crawl film_urls -o dataset/film_urls.json

In [9]:
%%writefile spiders/collect_data_films.py
import scrapy
import pandas as pd
import numpy as np
from selenium import webdriver

class collect_data_film(scrapy.Spider):
  name='data_film'
  def __init__(self):
    try:
      df = pd.read_json('./dataset/film_urls.json', encoding='utf-8-sig')
      self.urls=df['URL']
    except IOError:
      print("File not found")
  def start_requests(self):
    self.headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:48.0) Gecko/20100101 Firefox/48.0'}
    # CALOS
    # scrapy shell -s USER_AGENT='custom user agent' 'https://www.imdb.com/title/tt1321865/?ref_=ttls_li_tt'
    # JFK
    # scrapy shell -s USER_AGENT='custom user agent' 'https://www.imdb.com/title/tt0102138/?ref_=ttls_li_tt'
    # THE FIRM
    # scrapy shell -s USER_AGENT='custom user agent' 'https://www.imdb.com/title/tt0095158/?ref_=ttls_li_tt'
    for i in range(1000):
      next_page_url=f'https://www.imdb.com{self.urls[i]}?ref_=ttls_li_tt'
      yield scrapy.Request(url=next_page_url, callback=self.parse,headers=self.headers)

  def parse(self, response):

    # ID
    id_film=response.css('a.ipc-lockup-overlay.ipc-focusable::attr(href)').re(r'\w*\d{4,10}')[0]
    # Name
    name=response.css('h1[data-testid="hero-title-block__title"]::text').get()
    # Year release
    published_year=response.css("a[href*=releaseinfo]::text").re(r'\d{4}')[0]
    # Rate age
    age_rated=response.css("a[href*=certificates]::text").get()
    # Duration
    duration=response.css('div.ipc-metadata-list-item__content-container::text').getall()
    # Genres
    genres=response.css('span.ipc-chip__text::text').getall()

    # Director
    Director=set(response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]').xpath(".//*[contains(text(),'Director')]").xpath('following-sibling::div/ul/li/a/text()').getall())
    # Writes
    Writers=set(response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]').xpath(".//*[contains(text(),'Writer')]").xpath('following-sibling::div/ul/li/a/text()').getall())
    # Stars
    Stars=set(response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]').xpath(".//*[contains(text(),'Star')]").xpath('following-sibling::div/ul/li/a/text()').getall())
    
    # Budget
    try:
      Budget=response.xpath(".//*[contains(text(),'Budget')]").xpath('following-sibling::div/ul/li/label/text()').re(r'^.?.*\s')[0]
    except:
      Budget=None

    #  IMDb rating
    IMDb_rating = response.css("span.sc-7ab21ed2-1::text").get()
    # Popularity
    popularity = response.css("div.sc-edc76a2-1::text").get()
    # User reviews
    user_reviews = response.css("span.three-Elements:contains(User\ reviews) span::text").get()
    # Critic reviews
    critic_reviews = response.css("span.three-Elements:contains(Critic\ reviews) span::text").get()
    # Metascore
    meta_score = response.css("span.score-meta::text").get()
    # Oscar wins
    wins = response.css("a[href*=awards]::text").get()
    
    yield {
      'ID':id_film,
      'Name':name,
      'Published Year':published_year,
      'Rated':age_rated,
      'Duration':duration,
      'Genres':genres,
      
      'Director':Director,
      'Writers':Writers,
      'Stars':Stars,
      'Budget (estimated)':Budget,
      
      "IMDb RATING": IMDb_rating,
      "Popularity" : popularity,
      "User reviews": user_reviews,
      "Critic reviews": critic_reviews,
      "Meta score": meta_score,
      "Wins": wins,
    }

Overwriting spiders/collect_data_films.py


In [ ]:
!scrapy crawl data_film -o dataset/data_film.json

## <font color='orange'> Tiền xử lý dữ liệu </font> ##

<h4>1. Đọc dữ liệu</h4>

- <h5>Đọc file film_urls vào dataframe<h5>

In [20]:
df_url=pd.read_json('dataset/film_urls.json',encoding='utf-8-sig')
df_url.head()

,Top,ID,URL,Votes,Gross
0,1.,tt0068646,/title/tt0068646/,1848805,"134,966,411"
1,2.,tt0099685,/title/tt0099685/,1156484,"46,836,394"
2,3.,tt0110912,/title/tt0110912/,2042476,"107,928,762"
3,4.,tt0114814,/title/tt0114814/,1083052,"23,341,568"
4,5.,tt0078788,/title/tt0078788/,666680,"83,471,511"


- <h5>Đọc file data_film vào dataframe</h5>

In [31]:
df_film=pd.read_json('dataset/data_film.json',encoding='utf-8-sig')
df_film.head()

,ID,Name,Published Year,Rated,Duration,Genres,Director,Writers,Stars,Budget (estimated),IMDb RATING,Popularity,User reviews,Critic reviews,Meta score,Wins
0,tt0068646,The Godfather,1972,R,"[2, , hours, , 55, , minutes]","[Crime, Drama, Back to top]",[Francis Ford Coppola],"[Francis Ford Coppola, Mario Puzo]","[James Caan, Al Pacino, Marlon Brando]","$6,000,000",9.2,92,5.2K,193,100.0,Won 3 Oscars
1,tt0099685,Goodfellas,1990,R,"[2, , hours, , 25, , minutes]","[Biography, Crime, Drama, Back to top]",[Martin Scorsese],"[Martin Scorsese, Nicholas Pileggi]","[Ray Liotta, Joe Pesci, Robert De Niro]","$25,000,000",8.7,194,1.5K,163,90.0,Won 1 Oscar
2,tt0137523,Fight Club,1999,R,"[2, , hours, , 19, , minutes]","[Drama, Back to top]",[David Fincher],"[Jim Uhls, Chuck Palahniuk]","[Brad Pitt, Edward Norton, Meat Loaf]","$63,000,000",8.8,139,4.3K,373,66.0,Nominated for 1 Oscar
3,tt0078788,Apocalypse Now,1979,R,"[2, , hours, , 27, , minutes]","[Drama, Mystery, War, Back to top]",[Francis Ford Coppola],"[Francis Ford Coppola, John Milius, Michael Herr]","[Robert Duvall, Marlon Brando, Martin Sheen]","$31,500,000",8.5,307,1.3K,302,94.0,Won 2 Oscars
4,tt0117951,Trainspotting,1996,R,"[1, , hour, , 33, , minutes]","[Drama, Back to top]",[Danny Boyle],"[Irvine Welsh, John Hodge]","[Ewan McGregor, Jonny Lee Miller, Ewen Bremner]","£1,500,000",8.1,784,685,156,83.0,Nominated for 1 Oscar


- <h5>Hợp dữ liệu lại</h5>

In [45]:
film_info_df=df_url.merge(df_film, on="ID", how="outer")
film_info_df.shape

(1000, 20)

In [46]:
film_info_df.head()

,Top,ID,URL,Votes,Gross,Name,Published Year,Rated,Duration,Genres,Director,Writers,Stars,Budget (estimated),IMDb RATING,Popularity,User reviews,Critic reviews,Meta score,Wins
0,1,tt0068646,/title/tt0068646/,1848805,134966411.0,The Godfather,1972,R,"[2, , hours, , 55, , minutes]","[Crime, Drama, Back to top]",[Francis Ford Coppola],"[Francis Ford Coppola, Mario Puzo]","[James Caan, Al Pacino, Marlon Brando]","$6,000,000",9.2,92,5.2K,193,100.0,Won 3 Oscars
1,2,tt0099685,/title/tt0099685/,1156484,46836394.0,Goodfellas,1990,R,"[2, , hours, , 25, , minutes]","[Biography, Crime, Drama, Back to top]",[Martin Scorsese],"[Martin Scorsese, Nicholas Pileggi]","[Ray Liotta, Joe Pesci, Robert De Niro]","$25,000,000",8.7,194,1.5K,163,90.0,Won 1 Oscar
2,3,tt0110912,/title/tt0110912/,2042476,107928762.0,Pulp Fiction,1994,R,"[2, , hours, , 34, , minutes]","[Crime, Drama, Back to top]",[Quentin Tarantino],"[Roger Avary, Quentin Tarantino]","[John Travolta, Samuel L. Jackson, Uma Thurman]","$8,000,000",8.9,115,3.5K,303,94.0,Won 1 Oscar
3,4,tt0114814,/title/tt0114814/,1083052,23341568.0,The Usual Suspects,1995,R,"[1, , hour, , 46, , minutes]","[Crime, Drama, Mystery, Back to top]",[Bryan Singer],[Christopher McQuarrie],"[Chazz Palminteri, Gabriel Byrne, Kevin Spacey]","$6,000,000",8.5,425,1.4K,154,77.0,Won 2 Oscars
4,5,tt0078788,/title/tt0078788/,666680,83471511.0,Apocalypse Now,1979,R,"[2, , hours, , 27, , minutes]","[Drama, Mystery, War, Back to top]",[Francis Ford Coppola],"[Francis Ford Coppola, John Milius, Michael Herr]","[Robert Duvall, Marlon Brando, Martin Sheen]","$31,500,000",8.5,307,1.3K,302,94.0,Won 2 Oscars


- <h5>Ta thấy các cột như URL, Votes, Gross đứng đầu bảng thì rất khó quan sát, để dễ  quan sát và kiểm tra hơn, chúng ta cùng sắp xếp lại các cột nào</h5>

In [47]:
film_info_df=film_info_df[['Top','ID','Name','Published Year','Rated','Duration','Genres','Director','Writers','Stars','IMDb RATING','Budget (estimated)','Gross','Popularity','Votes','User reviews','Critic reviews','Meta score','Wins','URL']]

In [48]:
film_info_df.head()

,Top,ID,Name,Published Year,Rated,Duration,Genres,Director,Writers,Stars,IMDb RATING,Budget (estimated),Gross,Popularity,Votes,User reviews,Critic reviews,Meta score,Wins,URL
0,1,tt0068646,The Godfather,1972,R,"[2, , hours, , 55, , minutes]","[Crime, Drama, Back to top]",[Francis Ford Coppola],"[Francis Ford Coppola, Mario Puzo]","[James Caan, Al Pacino, Marlon Brando]",9.2,"$6,000,000",134966411.0,92,1848805,5.2K,193,100.0,Won 3 Oscars,/title/tt0068646/
1,2,tt0099685,Goodfellas,1990,R,"[2, , hours, , 25, , minutes]","[Biography, Crime, Drama, Back to top]",[Martin Scorsese],"[Martin Scorsese, Nicholas Pileggi]","[Ray Liotta, Joe Pesci, Robert De Niro]",8.7,"$25,000,000",46836394.0,194,1156484,1.5K,163,90.0,Won 1 Oscar,/title/tt0099685/
2,3,tt0110912,Pulp Fiction,1994,R,"[2, , hours, , 34, , minutes]","[Crime, Drama, Back to top]",[Quentin Tarantino],"[Roger Avary, Quentin Tarantino]","[John Travolta, Samuel L. Jackson, Uma Thurman]",8.9,"$8,000,000",107928762.0,115,2042476,3.5K,303,94.0,Won 1 Oscar,/title/tt0110912/
3,4,tt0114814,The Usual Suspects,1995,R,"[1, , hour, , 46, , minutes]","[Crime, Drama, Mystery, Back to top]",[Bryan Singer],[Christopher McQuarrie],"[Chazz Palminteri, Gabriel Byrne, Kevin Spacey]",8.5,"$6,000,000",23341568.0,425,1083052,1.4K,154,77.0,Won 2 Oscars,/title/tt0114814/
4,5,tt0078788,Apocalypse Now,1979,R,"[2, , hours, , 27, , minutes]","[Drama, Mystery, War, Back to top]",[Francis Ford Coppola],"[Francis Ford Coppola, John Milius, Michael Herr]","[Robert Duvall, Marlon Brando, Martin Sheen]",8.5,"$31,500,000",83471511.0,307,666680,1.3K,302,94.0,Won 2 Oscars,/title/tt0078788/


<h4>2. Bắt đầu xử lý dữ liệu</h4>

- <h5>Kiểm tra xem có dữ liệu trùng nhau không (<em>Ở đây ID mang tính định danh cho từng phim, ta sẽ kiểm tra xem thử có ID phim nào trùng nhau không</em>)</h5>

In [50]:
check_duplucate=set(film_info_df.ID.duplicated())
if True in check_duplucate:
    have_duplicated_rows = True
else:
    have_duplicated_rows = False

In [51]:
# TEST
assert have_duplicated_rows == False

<h4> Ý nghĩa mỗi dòng:</h4>

- Là thông tin chi tiết của các phim trong [top 1000 phim hay nhất mọi thời đại](https://www.imdb.com/list/ls006266261/), được [IMDb](https://www.imdb.com/?ref_=nv_home)  tổng hợp.


<h4>Ý nghĩa mỗi cột:</h4>

- Top: thể hiện vị thứ của phim trong danh sách
- ID: là duy nhất và mang tính định danh cho phim
- Name: tên của phim
- Published Year: Năm đầu tiên ra mắt phim
- Rated: 
- Duration: Thời gian phim
- Genres: Thể loại mà phim thuộc về
- Director: Đạo diễn sản xuất phim
- Writers: 
- Stars: Diễn viên nổi tiếng tham gia(ở đây nhóm chỉ lấy 1 vài người)
- IMDb RATING: Điểm đánh giá của IMDb về phim
- Budget (estimated): Kinh phí thực hiện phim (ước tính)
- Gross: Doanh thu của phim
- Popularity: Xếp hạng tổng quát
- Votes: Số lượt bình chọn
- User reviews: Số lượng nhận xét của người xem
- Critic reviews: Số lượng nhận xét của nhà phê bình
- Meta score:
- Wins: Giải mà phim đạt được trong các cuộc thi
- URL: địa chỉ truy cập vào trang web mô tả chi tiết của phim
- Votes: số lượt bình chọn cho phim
- Gross: Thu nhập của phim tính đến thời điểm hiện tại


- <h5>Ta thấy ở cột Top, Gross đang ở kiểu object, nhưng đúng là ở dạng numberic, ta thực hiện tiền xử lý về đúng kiểu dữ liệu</h5>

In [52]:
film_info_df.Top=film_info_df.Top.replace(',','',regex=True).replace('\.','',regex=True).astype(int)
film_info_df.Gross=film_info_df.Gross.replace([",",None],['',np.nan],regex=True).astype(float)

<h1>MỌI NGƯỜI XEM THỬ CÒN PHẦN NÀO CHƯA XỬ LÝ THÌ XỬ LÝ NHA</h1>

In [53]:
film_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Top                 1000 non-null   int64  
 1   ID                  1000 non-null   object 
 2   Name                1000 non-null   object 
 3   Published Year      1000 non-null   int64  
 4   Rated               997 non-null    object 
 5   Duration            1000 non-null   object 
 6   Genres              1000 non-null   object 
 7   Director            1000 non-null   object 
 8   Writers             1000 non-null   object 
 9   Stars               1000 non-null   object 
 10  IMDb RATING         1000 non-null   float64
 11  Budget (estimated)  895 non-null    object 
 12  Gross               937 non-null    float64
 13  Popularity          782 non-null    object 
 14  Votes               1000 non-null   int64  
 15  User reviews        963 non-null    object 
 16  Critic 